# 函数

## 提取函数签名

In [1]:
from inspect import signature

def foo(a, b=1, **c):
    pass

sig = signature(foo)
sig

<Signature (a, b=1, **c)>

In [2]:
[(name, param.default) for name, param in sig.parameters.items()]

[('a', inspect._empty), ('b', 1), ('c', inspect._empty)]

`inspect.Signature` 对象有个 bind 方法，可以把任意个参数绑定到签名中的形参上，所用的规则与实参到形参的匹配方式一样。框架可以使用这个方法在真正调用函数前验证函数：

In [3]:
args = {'a': 'aaa', 'b': 'bbb', 'x': 'xxx', 'y': 'yyy'}
bound_args = sig.bind(**args)
[(name, value) for name, value in bound_args.arguments.items()]

[('a', 'aaa'), ('b', 'bbb'), ('c', {'x': 'xxx', 'y': 'yyy'})]

In [4]:
del args['a']
try:
    sig.bind(**args)
except Exception as e:
    print(e)

missing a required argument: 'a'


## 支持函数式编程的模块

得益于 operator 和 functools 等模块的支持，可以编写函数式风格的 Python 代码。

### operator 模块

operator 模块为多个算术运算符提供了对应的函数，从而避免编写类似 `lambda a, b: a * b` 这种平凡的匿名函数：

In [5]:
from functools import reduce
from operator import mul

reduce(mul, range(1, 6))

120

#### itemgetter

operator 模块中还有一类函数，能替代从序列中取出元素或读取对象属性的 lambda 表达式：

In [6]:
from operator import itemgetter

data = [
    ('c', 2),
    ('b', 3), 
    ('a', 1)
]

sorted(data, key=itemgetter(0))

[('a', 1), ('b', 3), ('c', 2)]

如果把多个参数传给 itemgetter ，它构建的函数会返回提取的值构成的元组：

In [7]:
[itemgetter(1, 0)(t) for t in data]

[(2, 'c'), (3, 'b'), (1, 'a')]

#### attrgetter

attrgetter 与 itemgetter 作用类似，它创建的函数根据名称提取对象的属性。如果把多个属性名传给 attrgetter ，它也会返回提取的值构成的元组。

此外，如果参数名中包含 `.` ，attrgetter 会深入嵌套对象，获取指定的属性。

#### methodcall

methodcall 会自行创建函数，该函数会在对象上调用参数指定的方法：

In [8]:
from operator import methodcaller
s = "hello world"
upcase = methodcaller('upper')
upcase(s)

'HELLO WORLD'

In [9]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'hello-world'

## 闭包

闭包是一种函数，它会保留定义函数时存在的 **自由变量(free variable)** 的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

In [10]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager()
avg(10)
avg(11)
avg(12)

avg.__code__.co_varnames

('new_value', 'total')

In [11]:
avg.__code__.co_freevars

('series',)

`__closure__` 中的各个元素对应于 `__code__.co_freevars` 中的一个名称：

In [12]:
avg.__closure__

(<cell at 0x10b65d498: list object at 0x10b75b848>,)

In [13]:
avg.__closure__[0].cell_contents

[10, 11, 12]

### nonlocal

nonlocal 的作用是把变量标记为自由变量。

In [14]:
def make_averager():
    count = 0
    total = 0 
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(1)
avg(2)
avg(3)

2.0

### 闭包的陷阱

In [15]:
def count():
    fs = []
    for i in range(1, 4):
        def f():
             return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count()
f1(), f2(), f3()

(9, 9, 9)

需注意：<span style="color: red">闭包中不要引用任何可能会变化的变量。</span>如果一定要引用会变化的变量，可以再创建一个函数：

In [16]:
def count():
    def f(j):
        def g():
            return j*j
        return g
    fs = []
    for i in range(1, 4):
        fs.append(f(i))
    return fs

f1, f2, f3 = count()
f1(), f2(), f3()

(1, 4, 9)

## 装饰器

### functool.wraps

In [17]:
import functools
import time

def clock(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        t0 = time.perf_counter()
        result = func(*args, **kw)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kw:
            pairs = ['{}={}'.format(k, w) for k, w in sorted(kw.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print("[{:0.8f}] {}({}) -> {}".format(elapsed, name, arg_str, result))
        return result
    return wrapper

@clock
def snooze():
    time.sleep(1)

snooze()

[1.00004738] snooze() -> None


### functools.lru_cache

这是一项优化技术，它把耗时的函数的结果缓存起来，避免传入相同的参数时重复计算。lru_cache 可以使用两个可选的参数来配置：

``` python
functools.lru_cache(maxsize=128, typed=False)
```

maxsize 指定存储多少个调用的结果。缓存满了之后，旧的结果会被删除，腾出空间。为了得到最佳性能，maxsize 应设为 2 的幂。
typed 参数如果设为 True ，把不同参数类型得到的结果分开保存，即把通常认为相等的浮点数和整数参数(如 1 和 1.0)区分开。

因为 lru_cache 使用字典存储结果，而且键根据调用时传入的定位参数和关键字参数创建，因此被 lru_cache 装饰的函数，它的所有参数必须是可散列的。

In [18]:
@clock
def fib(n):
    if n < 2: return n
    return fib(n-2) + fib(n-1)

fib(6)

[0.00000068] fib(0) -> 0
[0.00000198] fib(1) -> 1
[0.00053346] fib(2) -> 1
[0.00000097] fib(1) -> 1
[0.00000101] fib(0) -> 0
[0.00000118] fib(1) -> 1
[0.00044184] fib(2) -> 1
[0.00194852] fib(3) -> 2
[0.00270590] fib(4) -> 3
[0.00000111] fib(1) -> 1
[0.00000115] fib(0) -> 0
[0.00000178] fib(1) -> 1
[0.00067502] fib(2) -> 1
[0.00094933] fib(3) -> 2
[0.00000105] fib(0) -> 0
[0.00000131] fib(1) -> 1
[0.00034278] fib(2) -> 1
[0.00000115] fib(1) -> 1
[0.00000124] fib(0) -> 0
[0.00000165] fib(1) -> 1
[0.00034642] fib(2) -> 1
[0.00061123] fib(3) -> 2
[0.00153016] fib(4) -> 3
[0.00311146] fib(5) -> 5
[0.00648946] fib(6) -> 8


8

In [19]:
import functools

@functools.lru_cache()
@clock
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

fib(6)

[0.00000126] fib(0) -> 0
[0.00000169] fib(1) -> 1
[0.00104439] fib(2) -> 1
[0.00000265] fib(3) -> 2
[0.00118195] fib(4) -> 3
[0.00000275] fib(5) -> 5
[0.00253019] fib(6) -> 8


8

### functools.singledispatch

使用 @singledispatch 装饰的普通函数会变成分派函数，或称为泛函数 (generic function) ：根据第一个参数的类型，选择对应的函数。

#### 分派函数

``` python
def handle_value(value):
    if isinstance(value, int):
        handle_value_int(value)
    elif isinstance(value, str):
        handle_value_str(value)
    else:
        handle_value_defalut(value)
```

@singledispatch 的优点是支持模块化扩展：各个模块可以为它支持的各个类型注册一个专门的函数。

In [20]:
from functools import singledispatch
from collections import abc
import numbers

@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)
    
@fun.register(numbers.Integral)
def _(arg, verbose=False):  # 函数名称无关紧要
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)
    
@fun.register(abc.MutableSequence)
@fun.register(tuple)  # 可以叠放多个 register 装饰器，以支持不同的类型
def handle_seq(arg, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

fun("3", True)

Let me just say, 3


In [21]:
fun(3, True)

Strength in numbers, eh? 3


In [22]:
fun((3,), True)

Enumerate this:
0 3


In [23]:
fun.dispatch(tuple)

<function __main__.handle_seq>

In [24]:
fun.registry.keys()

dict_keys([<class 'object'>, <class 'numbers.Integral'>, <class 'collections.abc.MutableSequence'>, <class 'tuple'>])

注册的专门函数应该处理抽象基类 (如 numbers.Integral 和 abc.MutableSequence) ，不要处理具体实现 (如 int 和 list) ，这样，代码支持的兼容类型更广泛。

## 参数

在 Python 中定义函数，可以用必选参数、默认参数、可变参数、关键字参数和命名关键字参数，这 5 种参数都可以组合使用。

组合参数定义的顺序必须是：必选参数、默认参数、可变参数、命名关键字参数和关键字参数。

### 命名关键字参数

命名关键字参数可以用于限制关键字参数的名字。命名关键字参数必须传入参数名，如果没有传入参数名，调用将报错。

In [25]:
# 只接收 city 和 job 作为关键字参数
def person(name, age, *, city, job):             # 命名关键字参数需要一个特殊分隔符 *
    print(name, age, city, job)                  # * 后面的参数被视为命名关键字参数

def person2(name, age, *, city='Beijing', job):  # 可以有默认值
    print(name, age, city, job)

# 如果函数定义中已经有了一个可变参数
# 后面跟着的命名关键字参数就不再需要一个特殊分隔符 * 了
def person(name, age, *args, city, job):
    print(name, age, args, city, job)

### 默认参数

<span style="color: red">定义默认参数时，默认参数必须指向不可变对象。</span>

In [26]:
def add_end(L=[]):
    L.append('END')
    return L

add_end()

['END']

In [27]:
add_end()

['END', 'END']

In [28]:
add_end()

['END', 'END', 'END']

原因：

函数在定义的时候，默认参数 L 的值就被计算出来了，即 `[]` ，且保存在函数对象中。每次调用该函数，如果改变了 L 的内容，则下次调用时，默认参数的内容就变了。

可以用 None 这个不可变对象来实现：

In [29]:
def add_end(L=None):
    if L is None:
        L = []
        L.append('END')
        return L